# Discrete problem: New
Jim Brouzoulis 2025

In [44]:
import sys
import os
# Add parent directory to sys.path
parent_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)
        
from mha021 import *

## Preprocessing step
define all input paramters and geometry

In [54]:
# Preprocessing step - define all input paramters and geometry in the beginning

k = 1 # spring stiffness same for all elements  [N/m]
P = 1 # external force [N]

# Edof - element degrees of freedom: lists what dofs each element has        
Edof = np.array([# dof1   dof2
                [   1,     2 ], # el 1
                [   1,     3 ], # el 2
                [   2,     4 ], # el 3
                [   2,     3 ], # el 4
                [   3,     4 ]  # el 5
                ])

num_dofs = np.max(Edof) # number of degrees of freedom
num_el = Edof.shape[0] # number of elements
print(f"number of dofs = {num_dofs}")
print(f"number of elements = {num_el}")

number of dofs = 4
number of elements = 5


## Processing
Assemble stiffness matrix, load vector and seting up the system of equations

In [55]:
# Initialise stiffness matrix and load vector
K = np.zeros((num_dofs, num_dofs)) # initialise stiffness matrix
f = np.zeros((num_dofs)) # initialise load vector

# Element stiffness matrices
# Example of spring1e - spring element, 1D, e- element stiffness
K1 = K2 = K3 = K4 = K5 = spring1e(k)

displayvar("K_1", K1) # looks the same for all elements

<IPython.core.display.Math object>

In [56]:
# First element
dofs1 = Edof[0, :] # pick dofs from the defined topology matrix Edof
displayvar("dofs 1", dofs1)
assem(K, K1, dofs1)
displayvar("K", K) # this is global K after first element has been assembled

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [57]:
# Second element
dofs2 = Edof[1, :]
displayvar("dofs 2", dofs2)
assem(K, K2, dofs2)
displayvar("K", K)

# Third element
dofs3 = Edof[2, :]
displayvar("dofs 3", dofs3)
assem(K, K3, dofs3)
displayvar("K", K)

# Forth element
dofs4 = Edof[3, :]
displayvar("dofs 4", dofs4)
assem(K, K4, dofs4)
displayvar("K", K)

# Forth element
dofs5 = Edof[4, :]
displayvar("dofs 5", dofs5)
assem(K, K5, dofs5)
displayvar("K", K)

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [58]:
# Alternative for assembling - This is the way!
k_vec = np.array([k, k, k, k, k])
K = np.zeros((num_dofs, num_dofs))
for el in range(num_el):
    Ke = spring1e(k_vec[el])
    dofs = Edof[el, :]
    assem(K, Ke, dofs)
    
displayvar("K", K)

<IPython.core.display.Math object>

### Boundary conditions
$a_1 = a_4 = 0$ and $f_2=P$

In [ ]:
# External loads
f[1] = P # acts in dof 2 => position 1 in the array f
# assem(f, [P], [2]) # you could also assemble this single value
displayvar("f", f)

# Support conditions
# bc -boundary conditions
bc_dofs = np.array([1, 4]) # prescribed dofs
bc_vals = np.array([0, 0]) # prescribed values


# Solving the system of equations
# we use the helper function solve_eq
a, r = solve_eq(K, f, bc_dofs, bc_vals)
# solution vector
displayvar("a", a)

# reaction forces
displayvar("r", r)

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

0.125

## Postprocessing step - spring forces
The normal force in each spring is computed as $N = k \, δ = k\, (a_2 - a_1)$

In [67]:
# Deformation of each element
# el 1

delta1 = a[dofs1[1]] - a[dofs1[0]] # a₂ - a₁ 
N1 = k * delta1


# print("Normal forces:", N1, N2, N3)

In [64]:
# We could also use the helper function to compute the spring forces
N = np.zeros(num_el)
for el in range(num_el):
    dofs = Edof[el, :]
    N[el] = spring1s(k, a[dofs-1])
print("Normal forces stored as a vector:", N)

# # N = spring1s(k, [a₁, a₂])
# N1 = spring1s(k1, [a[0], a[1]])
# N2 = spring1s(k2, [a[1], a[2]])
# N3 = spring1s(k3, [a[1], a[2]])
# print("Normal forces as separate calls:", N1, N2, N3)


Normal forces stored as a vector: [ 0.375  0.125 -0.375 -0.25  -0.125]
